In [10]:
import nidaqmx
import time
from nidaqmx.constants import LineGrouping

In [27]:
#Verify NIDAQ path (card_id in cell below)

import nidaqmx.system
# List all connected devices
system = nidaqmx.system.System.local()

for device in system.devices:
    print(f"Device Name: {device.name}")


Device Name: Dev1


In [31]:
#option1: set up digital trigger
#better for single pulse, simpler. but where is trigger mode referenced?
card_id = 'Dev1'
trigger_mode = 'out'

# Create a task
with nidaqmx.Task() as task:
    # Add a digital output channel
    task.do_channels.add_do_chan(f"{card_id}/port0/line1",name_to_assign_to_lines="2P3_Trigger")
    # Start the task
    task.start()

    # Start scan (send high signal)
    task.write(True)  # Send a high signal (1)
    
    # Wait for 1 millisecond
    time.sleep(0.001)
    
    # Send low signal
    task.write(False)  # Send a low signal (0)

    # Stop the task
    task.stop()


In [28]:
#option 2: set up digital trigger #both worked
#potentially better for sending multiple pulses
card = {
    'trigger_mode': 'out',
    'id': 'Dev1',
    'name': 'nidaq',
}

if card['trigger_mode'] == 'out':
    if 'dio' not in card:
        # Create a digital output task
        card['dio'] = nidaqmx.Task()
        # Add a digital output channel to Port0/Line1 with proper LineGrouping
        card['dio'].do_channels.add_do_chan(
            'Dev1/port0/line1', 
            line_grouping=LineGrouping.CHAN_PER_LINE)

# Start scan
with card['dio'] as task:
    # Write a digital high signal (1) as a boolean array
    task.write([True], auto_start=True)

    data_start_time = time.time()  # Record start time
    time.sleep(0.001)  # Wait for 1 ms

    # Write a digital low signal (0)
    task.write([False], auto_start=True)


### on atlas pc make sure to run 

conda install conda-forge::nidaqmx-python

conda install conda-forge::python-decouple

pip install hightime

may need to install NI MAX app. 

### On 2P3: 
make sure to select start with input trigger AND to press start T-Series
create a env style to save and import at the start of each session to maintain exactly the same params!! and importantly the image dimensions, frame rate, save type, location, etc. 


## AGHGHHHH
### tips to troubleshoot send trigger 

Open NI MAX app. Or install it if not installed. Check 'Devices and Interfaces' dropdown. Check Name and run Self Tests. If it fails, soemthing else might be actively occupying the port. 

In my case, MATLAB was actively occupying the path bc the self-test did not fail after I closed MATLAB and this script properly sent trigger :)